In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import glob
import pickle

from music21 import converter, instrument, stream, note, chord
import tensorflow as tf

#Run version 2.1.6
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

In [5]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)


In [6]:
VOCAB = (len(set(notes)))


In [7]:
def build_model(lstm_input):
    '''
    Build and compile the model
    
    lstm_input: lstm_input.shape[1] = number of steps, lstm_input.shape[2] = number features needed for
    Bi-directional
    
    model_output: number of categories to output for classification
    
    returns: compiled model
    '''
    
    model = Sequential()
    model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    #model.add(SeqSelfAttention(attention_width=15, attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    # model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=True)))
    # #model.add(SeqSelfAttention(attention_width=15, attention_activation='sigmoid'))
    # model.add(Dropout(0.3))
    
    # model.add(Bidirectional(LSTM(512,input_shape=(lstm_input.shape[1], lstm_input.shape[2]), return_sequences=False)))
    # #model.add(SeqSelfAttention(attention_width=15, attention_activation='sigmoid'))
    # model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(VOCAB))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    model.load_weights('first_train.h5')
    
    
    return model

In [8]:
def prepare_notes(notes, pitches, VOCAB):
    '''
    prepare lstm input notes again used by the network to predict notes
    
    '''
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_normalized = lstm_input / float(len(pitches))
    
    return lstm_input, lstm_normalized

In [46]:
def predict_to_notes(model, lstm_input, pitches, VOCAB):
    '''
    **Generated predictions from model based on random starting point**
    
    model: --> Original model with weights loaded
    lstm_inputL: --> Output from prepare, used to initialize pattern with an int the model recognizes
    pitches: ---> list of all the notes, chords and rests
    VOCAB:----> Number of unique notes to classify = len(pitches)
    
    output: Predicted_notes 
    '''
    #random starting point 
    start = np.random.randint(0, len(lstm_input) -1)
    
    note_dict = dict((number, note) for number, note in enumerate(pitches))
    
    pattern = lstm_input[start]
    print(type(pattern))
    predicted_notes = []
    
    for note in range(500):
        to_predict = np.reshape(pattern, (1, len(pattern), 1))
        to_predict = to_predict/ float(VOCAB)
        
        prediction = model.predict(to_predict, verbose = 0)
        print(prediction)
        
        index = np.argmax(prediction)
        print (index)
        result = note_dict[index]
        predicted_notes.append(result)
        
        np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
        
    return predicted_notes
    

In [47]:
def midi_convert(predicted_notes):
    '''
    convert the notes in predicted_notes to midi files
    
    predicted_notes: Output from function predict_to_notes() --> list of predicted notes
    
    returns: None --- > Creates a midi file when ran
    '''
    offset = 0 
    midi_notes = []
    
    #create notes, chords, and rest objects from predicted_notes
    for pattern in predicted_notes:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        #checking to see if a note is a chord 
        if ('.' in pattern) or pattern.isdigit():
            chord = pattern.split('.')
            notes = []
            for current_note in chord:
                this_note = note.Note(int(current_note))
                this_note.storedInstrument = instrument.Piano()
                notes.append(this_note)
            new_chord = chord.Chord(notes) 
            new_chord.offset = offset
            midi_notes.append(new_chord)
        #if the pattern is a rest    
        elif ('rest' in pattern):
            this_rest = note.Rest(pattern)
            this_rest.offset = offset
            this_rest.storedInstrument = instrument.Piano() #Still needs to be paino instrument even though = rest
            midi_notes.append(this_rest)
        else:
            this_note = note.Note(pattern)
            this_notes.offset = offset 
            this_notes.storedInstrument = instrument.Piano()
            midi_notes.append(this_note)
        #ensure that the notes do not stack    
        offset += convert_to_float(duration)
    
    midi = stream.Stream(midi_notes)
    midi.write('midi', fp = 'predicted_song.mid')
    
    
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [44]:
def generate():
    '''
    generates the midi file
    
    
    
    output:None ---> creates the midi file
    
    '''
    #Load the notes
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    
    #create the pitchnames
    pitches = sorted(set(note for note in notes))
    
    
    lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB)
    model = keras.models.load_model('first_model.h5')
    predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)
    midi_convert(predicted_notes)
    

In [23]:
generate()

[[9.6668273e-06 9.6840731e-06 1.7728460e-05 ... 7.2345288e-06
  1.0579193e-05 6.2293907e-06]]
2100


KeyError: 2100

In [48]:
pitches = sorted(set(note for note in notes))
lstm_input, lstm_normalized = prepare_notes(notes, pitches, VOCAB)
predicted_notes = predict_to_notes(model, lstm_input, pitches, VOCAB)

<class 'numpy.ndarray'>
[[1.4239976e-05 1.2925325e-05 2.1705460e-05 ... 1.3703597e-05
  2.0298661e-05 1.1796689e-05]]
359


ValueError: in user code:

    /Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    

In [25]:

model = keras.models.load_model('first_model.h5')

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 100, 1024)         2105344   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2424)              248220024 
_________________________________________________________________
activation_2 (Activation)    (None, 2424)              0         
Total params: 250,325,368
Trainable params: 250,325,368
Non-trainable params: 0
_________________________________________________________________


In [35]:
pitches = sorted(set(note for note in notes))

['0.1.3.7.81.0',
 '0.1.5.80.25',
 '0.1.5.80.5',
 '0.1.5.81.0',
 '0.1.50.75',
 '0.1.60.5',
 '0.1.62.5',
 '0.10.0',
 '0.2.3.5.70.5',
 '0.2.4.70.5',
 '0.2.4.71.0',
 '0.2.40.5',
 '0.2.5.70.5',
 '0.2.50.5',
 '0.2.52/3',
 '0.2.60.25',
 '0.2.60.5',
 '0.2.61/3',
 '0.2.62/3',
 '0.2.64/3',
 '0.2.70.5',
 '0.2.70.75',
 '0.2.71.0',
 '0.2.71/3',
 '0.20.0',
 '0.20.25',
 '0.20.5',
 '0.20.75',
 '0.21.0',
 '0.21/3',
 '0.22/3',
 '0.3.50.5',
 '0.3.51/3',
 '0.3.52/3',
 '0.3.6.91/3',
 '0.3.6.94.0',
 '0.3.60.5',
 '0.3.62/3',
 '0.3.70.25',
 '0.3.70.5',
 '0.3.70.75',
 '0.3.71.0',
 '0.3.71.5',
 '0.3.71/3',
 '0.30.25',
 '0.30.5',
 '0.30.75',
 '0.31.0',
 '0.31.5',
 '0.31.75',
 '0.31/3',
 '0.33.0',
 '0.34/3',
 '0.4.52.0',
 '0.4.60.5',
 '0.4.60.75',
 '0.4.70.25',
 '0.4.70.5',
 '0.4.70.75',
 '0.4.71.0',
 '0.4.71.75',
 '0.4.71/3',
 '0.4.72.0',
 '0.4.72.5',
 '0.4.72.75',
 '0.4.72/3',
 '0.4.74.0',
 '0.4.80.25',
 '0.40.25',
 '0.40.5',
 '0.40.75',
 '0.41.0',
 '0.41.5',
 '0.41.75',
 '0.41/3',
 '0.410/3',
 '0.42.0',
 '0.42

In [39]:
len(note_dict)

2424

In [38]:
note_dict = dict((number, note) for number, note in enumerate(pitches))
    